In [1]:
import torch
import numpy as np
from sae_lens import SAE, HookedSAETransformer
import pandas as pd
from torch import nn
import ezkl
import os
import json
import time
import time
from multiprocessing import Pool, cpu_count
import numpy as np

#from sae_lens.toolkit.neuronpedia_integration import get_feature_from_neuronpedia


/ext3/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FreivaldsVerificationSAE(SAE):
    def __init__(self, cfg, features_to_check=None, num_trials=10, tol=0.01):
        super().__init__(cfg)
        self.features_to_check = [2592, 4445, 4663, 4733, 6531, 8179, 9566, 20927, 24185]   # List of feature indices to check
        self.num_trials = num_trials
        self.tol = tol
        self.feature_present = False
        

    def encode_standard(self, x):
        # Use the original encode function
        feature_acts = super().encode_standard(x)

        print(feature_acts.shape)

        # Store the feature activations for later verification
        self._last_feature_acts = feature_acts

        for feature_idx in self.features_to_check:
            # Create a copy of feature activations with the target feature zeroed out
            modified_acts = feature_acts.clone()
            modified_acts[:, :, feature_idx] = 0
    
        
        return feature_acts
    
    def decode(self, feature_acts):
        # Use the original decode function to get the reconstruction
        sae_out = super().decode(feature_acts)

        sae_out_preact = self.apply_finetuning_scaling_factor(feature_acts) @ self.W_dec + self.b_dec
        # print("preact shape", sae_out_preact.shape)
        
        # Verify specific features using Freivalds' algorithm
        # if self.features_to_check is not None:
        
        self.verify_features(feature_acts, sae_out_preact)
        
            
        return sae_out
    
    def verify_features(self, feature_acts, sae_out_preact):
        """Verifies if specific features contribute to the reconstruction using Freivalds' algorithm."""
        batch_size, seq_len, d_sae = feature_acts.shape
        d_in = self.W_dec.shape[-1]
        # print("Preact shape", sae_out_preact.shape)
        
        
        for feature_idx in self.features_to_check:
            # Create a copy of feature activations with the target feature zeroed out
            modified_acts = feature_acts.clone()
            modified_acts[:, :, feature_idx] = 0
            
            # Check if reconstruction changes significantly using Freivalds
            self.feature_present += self.freivalds_verify_reconstruction(
                feature_acts, modified_acts, sae_out_preact
            )
        

    def freivalds_verify_reconstruction(self, original_acts, modified_acts, sae_out_preact, num_trials=None, tol=None):
        """
        Freivalds' verification integrated into SAE class
        """
        d_in = self.W_dec.shape[-1]
        self.pre_queried_random_vectors = [
            torch.randint(0, 2, (d_in, 1), 
                      dtype=modified_acts.dtype, 
                      device=modified_acts.device)
            for _ in range(self.num_trials)
        ]

        # batch_size, seq_len, d_sae = feature_acts.shape
        # print("Shape SAE out:", sae_out_preact.shape)
        # print("Shape Preact SAE out:", sae_out_preact.shape)
        # print("Shape Decoder Bias:", self.b_dec.shape)
        # print("Shape Feature activations:", self.apply_finetuning_scaling_factor(feature_acts).shape)
        # print("Shape Decoder Weights:", self.W_dec.shape)
        # print(f'Seeking:{sae_out_preact - self.b_dec - self.apply_finetuning_scaling_factor(feature_acts) @ self.W_dec}')

        ## Non parallelized
        start_time = time.perf_counter()
        
        for i in range(self.num_trials):
            r = self.pre_queried_random_vectors[i]
            
            # Compute both sides of equation
            Cr = (sae_out_preact - self.b_dec) @ r
            Br = self.W_dec @ r
            ABr = self.apply_finetuning_scaling_factor(modified_acts) @ Br
            # print("Original", torch.norm((self.apply_finetuning_scaling_factor(original_acts) @ Br) - Cr))
            # print("Modified", torch.norm(ABr - Cr))
            
            if torch.norm(ABr - Cr, p=float('inf')) > self.tol:
                end_time = time.perf_counter()
                freivalds_time = end_time - start_time
                print(f"Freivalds verification True: {freivalds_time:.6f} seconds")
                return True
            else:
                end_time = time.perf_counter()
                freivalds_time = end_time - start_time
                print(f"Freivalds verification False: {freivalds_time:.6f} seconds")
        return False



        ## Parallelized ## Will Require HF Parallelism diabling
        # def verify_trial(args):
        #     i, vectors_list, W_dec_np, b_dec_np, scaled_acts_np, sae_out_preact_np, tol = args
            
        #     # Get the pre-generated random vector for this trial
        #     r_np = vectors_list[i]
            
        #     # Compute both sides of equation with NumPy
        #     Cr_np = (sae_out_preact_np - b_dec_np) @ r_np
        #     Br_np = W_dec_np @ r_np
        #     ABr_np = scaled_acts_np @ Br_np
            
        #     # Calculate norm using NumPy
        #     norm_val = np.max(np.abs(ABr_np - Cr_np))
        #     return norm_val > tol
        
        # """
        # Parallelized Freivalds' verification using multiprocessing
        # """
        # # Apply scaling factor before converting to NumPy
        # scaled_acts = self.apply_finetuning_scaling_factor(modified_acts)
        
        # # Convert PyTorch tensors to NumPy arrays for multiprocessing
        # scaled_acts_np = scaled_acts.cpu().detach().numpy()
        # sae_out_preact_np = sae_out_preact.cpu().detach().numpy()
        # W_dec_np = self.W_dec.cpu().detach().numpy()
        # b_dec_np = self.b_dec.cpu().detach().numpy()
        # vectors_list = [r.cpu().detach().numpy() for r in self.pre_queried_random_vectors]
        
        # start_time = time.perf_counter()
        
        # args = [(i, vectors_list, W_dec_np, b_dec_np, scaled_acts_np, sae_out_preact_np, self.tol) 
        #         for i in range(self.num_trials)]
        
        # with Pool(processes=cpu_count()) as pool:
        #     results = pool.map(verify_trial, args)
            
        # end_time = time.perf_counter()
        # freivalds_time = end_time - start_time
        
        # if any(results):
        #     print(f"Freivalds verification True: {freivalds_time:.6f} seconds")
        #     return True
        # else:
        #     print(f"Freivalds verification False: {freivalds_time:.6f} seconds")
        #     return False


In [3]:
model = HookedSAETransformer.from_pretrained("gpt2-small", device="cpu")

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = FreivaldsVerificationSAE.from_pretrained(
    release="gpt2-small-res-jb",  # <- Release name
    sae_id="blocks.7.hook_resid_pre",  # <- SAE id (not always a hook point!)
    device="cpu",
)

Loaded pretrained model gpt2-small into HookedTransformer


/ext3/miniforge3/lib/python3.12/site-packages/sae_lens/sae.py:151: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [33]:
print(sae.cfg.__dict__)

{'architecture': 'standard', 'd_in': 768, 'd_sae': 24576, 'activation_fn_str': 'relu', 'apply_b_dec_to_input': True, 'finetuning_scaling_factor': False, 'context_size': 128, 'model_name': 'gpt2-small', 'hook_name': 'blocks.7.hook_resid_pre', 'hook_layer': 7, 'hook_head_index': None, 'prepend_bos': True, 'dataset_path': 'Skylion007/openwebtext', 'dataset_trust_remote_code': True, 'normalize_activations': 'none', 'dtype': 'torch.float32', 'device': 'cpu', 'sae_lens_training_version': None, 'activation_fn_kwargs': {}, 'neuronpedia_id': 'gpt2-small/7-res-jb', 'model_from_pretrained_kwargs': {'center_writing_weights': True}, 'seqpos_slice': (None,)}


In [6]:
print(model)  # Or:
print(model.cfg)

HookedSAETransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
     

In [7]:
from datasets import load_dataset
from transformer_lens.utils import tokenize_and_concatenate

dataset = load_dataset(
    path="NeelNanda/pile-10k",
    split="train",
    streaming=False,
)

token_dataset = tokenize_and_concatenate(
    dataset=dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=sae.cfg.context_size,
    add_bos_token=sae.cfg.prepend_bos,
)

Map: 100%|██████████| 10000/10000 [00:20<00:00, 490.98 examples/s]


In [8]:
import requests

url = "https://www.neuronpedia.org/api/explanation/export?modelId=gpt2-small&saeId=7-res-jb"
headers = {"Content-Type": "application/json"}

response = requests.get(url, headers=headers)

In [9]:
# convert to pandas
data = response.json()
explanations_df = pd.DataFrame(data)
# rename index to "feature"
explanations_df.rename(columns={"index": "feature"}, inplace=True)
# explanations_df["feature"] = explanations_df["feature"].astype(int)
explanations_df["description"] = explanations_df["description"].apply(
    lambda x: x.lower()
)
explanations_df

,modelId,layer,feature,description,explanationModelName,typeName
0,gpt2-small,7-res-jb,218,stars and dashed for censoring expletives,None,oai_token-act-pair
1,gpt2-small,7-res-jb,218,stars and dashes for censoring expletives,None,oai_token-act-pair
2,gpt2-small,7-res-jb,218,offensive language and expletives,None,oai_token-act-pair
3,gpt2-small,7-res-jb,2020,names of people,gpt-3.5-turbo,oai_token-act-pair
4,gpt2-small,7-res-jb,3493,references to nazism,gpt-3.5-turbo,oai_token-act-pair
...,...,...,...,...,...,...
24568,gpt2-small,7-res-jb,24571,locations and cities paired with information s...,gpt-3.5-turbo,oai_token-act-pair
24569,gpt2-small,7-res-jb,24572,"actions related to personal grooming, such as ...",gpt-3.5-turbo,oai_token-act-pair
24570,gpt2-small,7-res-jb,24573,"words containing the sequence ""lo""",gpt-3.5-turbo,oai_token-act-pair
24571,gpt2-small,7-res-jb,24574,instances of added or inserted text,gpt-3.5-turbo,oai_token-act-pair


In [10]:
bible_features = explanations_df.loc[explanations_df.description.str.contains(" bible")]
bible_features

,modelId,layer,feature,description,explanationModelName,typeName
11902,gpt2-small,7-res-jb,11735,"references to religious texts, particularly th...",gpt-3.5-turbo,oai_token-act-pair


In [4]:
from transformer_lens.utils import test_prompt

prompt = "How do i live my life"
promp2 = "Today is weekend, tomorrow is"
answer = "Monday"

# Show that the model can confidently predict the next token.
test_prompt(promp2, answer, model)

Tokenized prompt: ['<|endoftext|>', 'Today', ' is', ' weekend', ',', ' tomorrow', ' is']
Tokenized answer: [' Monday']


Performance on answer token:
Rank: 0        Logit: 13.96 Prob:  8.18% Token: | Monday|

Top 0th token. Logit: 13.96 Prob:  8.18% Token: | Monday|
Top 1th token. Logit: 13.85 Prob:  7.39% Token: | Saturday|
Top 2th token. Logit: 13.67 Prob:  6.15% Token: | Sunday|
Top 3th token. Logit: 13.63 Prob:  5.89% Token: | weekend|
Top 4th token. Logit: 13.34 Prob:  4.43% Token: | the|
Top 5th token. Logit: 12.85 Prob:  2.70% Token: | week|
Top 6th token. Logit: 12.70 Prob:  2.34% Token: | day|
Top 7th token. Logit: 12.67 Prob:  2.27% Token: | Friday|
Top 8th token. Logit: 12.58 Prob:  2.07% Token: | holiday|
Top 9th token. Logit: 12.56 Prob:  2.02% Token: | a|


Ranks of the answer tokens: [(' Monday', 0)]

In [5]:
sae.use_error_term

False

In [6]:
# hooked SAE Transformer will enable us to get the feature activations from the SAE
sae.feature_present = 0
_, cache = model.run_with_cache_with_saes(promp2, saes=[sae])

print([(k, v.shape) for k, v in cache.items() if "sae" in k])

# note there were 11 tokens in our prompt, the residual stream dimension is 768, and the number of SAE features is 768

torch.Size([1, 7, 24576])
Freivalds verification False: 0.005411 seconds
Freivalds verification False: 0.009720 seconds
Freivalds verification False: 0.013746 seconds
Freivalds verification False: 0.017730 seconds
Freivalds verification False: 0.021717 seconds
Freivalds verification False: 0.025689 seconds
Freivalds verification False: 0.029657 seconds
Freivalds verification False: 0.033612 seconds
Freivalds verification False: 0.037578 seconds
Freivalds verification False: 0.041543 seconds
Freivalds verification False: 0.003775 seconds
Freivalds verification False: 0.007568 seconds
Freivalds verification False: 0.011322 seconds
Freivalds verification False: 0.015075 seconds
Freivalds verification False: 0.018836 seconds
Freivalds verification False: 0.022569 seconds
Freivalds verification False: 0.026321 seconds
Freivalds verification False: 0.030066 seconds
Freivalds verification False: 0.033811 seconds
Freivalds verification False: 0.039085 seconds
Freivalds verification False: 0.00

In [67]:
sae.feature_present

2

In [75]:
class DirectVerificationSAE(SAE):
    def __init__(self, cfg, features_to_check=None, num_trials=10, tol=0.01):
        super().__init__(cfg)
        self.features_to_check = [2592, 4445, 4663, 4733, 6531, 8179, 9566, 20927, 24185]   # List of feature indices to check
        self.num_trials = num_trials
        self.tol = tol
        self.feature_present = False
        

    def encode_standard(self, x):
        # Use the original encode function
        feature_acts = super().encode_standard(x)

        print(feature_acts.shape)

        # Store the feature activations for later verification
        self._last_feature_acts = feature_acts

        for feature_idx in self.features_to_check:
            # Create a copy of feature activations with the target feature zeroed out
            modified_acts = feature_acts.clone()
            modified_acts[:, :, feature_idx] = 0
    
        
        return feature_acts
    
    def decode(self, feature_acts):
        # Use the original decode function to get the reconstruction
        sae_out = super().decode(feature_acts)

        sae_out_preact = self.apply_finetuning_scaling_factor(feature_acts) @ self.W_dec + self.b_dec
        # print("preact shape", sae_out_preact.shape)
        
        # Verify specific features using Freivalds' algorithm
        # if self.features_to_check is not None:
        
        self.verify_features(feature_acts, sae_out_preact)
        
            
        return sae_out
    
    def verify_features(self, feature_acts, sae_out_preact):
        """Verifies if specific features contribute to the reconstruction using Freivalds' algorithm."""
        batch_size, seq_len, d_sae = feature_acts.shape
        d_in = self.W_dec.shape[-1]
        # print("Preact shape", sae_out_preact.shape)
        
        
        for feature_idx in self.features_to_check:
            # Create a copy of feature activations with the target feature zeroed out
            modified_acts = feature_acts.clone()
            modified_acts[:, :, feature_idx] = 0
            
            # Check if reconstruction changes significantly using Freivalds
            self.feature_present += self.regular_verify_reconstruction(
                feature_acts, modified_acts, sae_out_preact
            )
        

    def regular_verify_reconstruction(self, original_acts, modified_acts, sae_out_preact, num_trials=None, tol=None):
        """
        Regular verification integrated into SAE class
        """
        d_in = self.W_dec.shape[-1]

        # batch_size, seq_len, d_sae = feature_acts.shape
        # print("Shape SAE out:", sae_out_preact.shape)
        # print("Shape Preact SAE out:", sae_out_preact.shape)
        # print("Shape Decoder Bias:", self.b_dec.shape)
        # print("Shape Feature activations:", self.apply_finetuning_scaling_factor(feature_acts).shape)
        # print("Shape Decoder Weights:", self.W_dec.shape)
        # print(f'Seeking:{sae_out_preact - self.b_dec - self.apply_finetuning_scaling_factor(feature_acts) @ self.W_dec}')
        start_time = time.perf_counter()
        
        for i in range(self.num_trials):
            
            # Compute both sides of equation
            C = (sae_out_preact - self.b_dec)
            AB = self.apply_finetuning_scaling_factor(modified_acts) @ self.W_dec
            # print("Original", torch.norm((self.apply_finetuning_scaling_factor(original_acts) @ Br) - Cr))
            # print("Modified", torch.norm(ABr - Cr))
            
            if torch.norm(AB - C, p=float('inf')) > self.tol:
                end_time = time.perf_counter()
                freivalds_time = end_time - start_time
                print(f"Normal verification True: {freivalds_time:.6f} seconds")
                return True
            else:
                end_time = time.perf_counter()
                freivalds_time = end_time - start_time
                print(f"Normal verification False: {freivalds_time:.6f} seconds")
                return False


In [76]:
model = HookedSAETransformer.from_pretrained("gpt2-small", device="cpu")

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = DirectVerificationSAE.from_pretrained(
    release="gpt2-small-res-jb",  # <- Release name
    sae_id="blocks.7.hook_resid_pre",  # <- SAE id (not always a hook point!)
    device="cpu",
)

Loaded pretrained model gpt2-small into HookedTransformer


In [78]:
sae.feature_present = 0
_, cache = model.run_with_cache_with_saes(promp2, saes=[sae])

torch.Size([1, 7, 24576])
Normal verification False: 0.005809 seconds
Normal verification False: 0.005704 seconds
Normal verification False: 0.005648 seconds
Normal verification False: 0.005600 seconds
Normal verification False: 0.005579 seconds
Normal verification False: 0.005529 seconds
Normal verification True: 0.005388 seconds
Normal verification True: 0.005364 seconds
Normal verification False: 0.005509 seconds


In [79]:
sae.feature_present

2

In [21]:
model = HookedSAETransformer.from_pretrained("gpt2-small", device="cpu")

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = TimedVerificationSAE.from_pretrained(
    release="gpt2-small-res-jb",  # <- Release name
    sae_id="blocks.7.hook_resid_pre",  # <- SAE id (not always a hook point!)
    device="cpu",
)

Loaded pretrained model gpt2-small into HookedTransformer


In [29]:
import time
import numpy as np

prompt = "Today is weekend, tomorrow is"
num_runs = 100
generation_times = []

for _ in range(num_runs):
    # Start timing for generation
    start_time = time.perf_counter()
    
    # Generate a response
    input_ids = model.tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_new_tokens=1)
    response = model.tokenizer.decode(output[0][-1:])
    
    # End timing for generation
    end_time = time.perf_counter()
    generation_times.append(end_time - start_time)

# Calculate average and standard deviation
average_generation_time = np.mean(generation_times)
std_dev = np.std(generation_times)

print(f"Average generation time: {average_generation_time:.6f} seconds")
print(f"Standard deviation: {std_dev:.6f} seconds")


100%|██████████| 1/1 [00:00<00:00,  7.75it/s]

Average generation time: 0.141705 seconds
Standard deviation: 0.068625 seconds


In [28]:
prompt = "Today is weekend, tomorrow is"

start_time1, end_time1 = 0, 0
# Start timing for generation
start_time1 = time.perf_counter()

# Generate a response
input_ids = model.tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_new_tokens=1)
response = model.tokenizer.decode(output[0][-1:])

# End timing for generation
end_time1 = time.perf_counter()
generation_time = end_time1 - start_time1

# Regex pattern to match feature
weekday_pattern = re.compile(r"^(mon|monday|tue|tues|tuesday|wed|wednesday|thu|thurs|thursday|fri|friday|sat|saturday|sun|sunday)$", re.IGNORECASE)

# Start timing for generation
start_time = time.perf_counter()

# Check if the response is a weekday
is_weekday = bool(weekday_pattern.match(response.strip()))

# End timing for generation
end_time = time.perf_counter()
verif_time = end_time - start_time

print(f"Prompt: {prompt}")
print(f"Model response: {response}")
print(f"Is weekday: {is_weekday}")
print(f"Generation time (seconds): {generation_time:.6f}")
print(f"Verification time (seconds): {verif_time:.6f}")


100%|██████████| 1/1 [00:00<00:00,  5.99it/s]

Prompt: Today is weekend, tomorrow is
Model response:  Saturday
Is weekday: True
Generation time (seconds): 0.170548
Verification time (seconds): 0.000050


In [13]:
import time
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load RoBERTa tokenizer and model for classification
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
classifier = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
classifier.eval()  # Set model to evaluation mode

# Define the prompt
prompt = "Today is weekend, tomorrow is"

# Start timing for generation
start_gen_time = time.perf_counter()

# Generate a response
input_ids = model.tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_new_tokens=1)
response = model.tokenizer.decode(output[0][-1:])

# End timing for generation
end_gen_time = time.perf_counter()
generation_time = end_gen_time - start_gen_time

# Define function to classify weekday
def classify_weekday(text):
    # Tokenize input
    inputs = tokenizer_roberta(text, return_tensors='pt')
    
    # Run model
    with torch.no_grad():
        outputs = classifier(**inputs)
    
    # For demonstration, we simulate classification since the model isn't fine-tuned for weekdays
    weekdays = ['mon', 'monday', 'tue', 'tues', 'tuesday', 'wed', 'wednesday', 
               'thu', 'thurs', 'thursday', 'fri', 'friday']
    
    if any(day in text.lower() for day in weekdays):
        simulated_logits = torch.tensor([[0.1, 10.0]])  # High confidence for weekday
    else:
        simulated_logits = torch.tensor([[10.0, 0.1]])  # High confidence for not weekday
    
    probs = torch.softmax(simulated_logits, dim=1)
    predicted_label = torch.argmax(probs, dim=1).item()
    
    return predicted_label == 1, probs[0, predicted_label].item()

# Start timing for classification
start_class_time = time.perf_counter()

# Classify the response
is_weekday, confidence = classify_weekday(response)

# End timing for classification
end_class_time = time.perf_counter()
classification_time = end_class_time - start_class_time

print(f"Prompt: {prompt}")
print(f"Model response: {response}")
print(f"Is weekday: {is_weekday}")
print(f"Confidence: {confidence:.4f}")
print(f"Generation time (seconds): {generation_time:.6f}")
print(f"Classification time (seconds): {classification_time:.6f}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1/1 [00:00<00:00,  4.06it/s]


Prompt: Today is weekend, tomorrow is
Model response:  holiday
Is weekday: False
Confidence: 0.9999
Generation time (seconds): 0.249698
Classification time (seconds): 1.048472


In [30]:
# Complete cell to gather computational environment details for research paper
import os
import sys
import json
import platform
import datetime
from IPython import get_ipython
import pkg_resources

# System information
system_info = {
    "python_version": sys.version,
    "platform": platform.platform(),
    "processor": platform.processor(),
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# Get installed packages (focusing on common data science libraries)
packages = {}
for pkg in pkg_resources.working_set:
    packages[pkg.key] = pkg.version

key_packages = {}
for pkg in ['numpy', 'pandas', 'matplotlib', 'tensorflow', 'torch', 'scikit-learn', 
            'jupyter', 'ipykernel', 'nbformat']:
    if pkg in packages:
        key_packages[pkg] = packages[pkg]

# Try to get Jupyter config information
jupyter_info = {}
try:
    from jupyter_core.paths import jupyter_config_dir
    jupyter_info["config_dir"] = jupyter_config_dir()
except:
    jupyter_info["config_dir"] = "Unable to retrieve"

# Try to get hardware utilization
hardware_info = {}
try:
    import psutil
    hardware_info = {
        "cpu_count": psutil.cpu_count(),
        "cpu_percent": psutil.cpu_percent(),
        "memory_total_gb": round(psutil.virtual_memory().total / (1024**3), 2),
        "memory_used_percent": psutil.virtual_memory().percent
    }
except:
    hardware_info = {"status": "psutil not available"}

# Try to get GPU information
gpu_info = {}
try:
    import GPUtil
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu_info = [{
            "name": gpu.name,
            "memory_total_mb": gpu.memoryTotal,
            "utilization_percent": round(gpu.load*100, 1),
            "memory_used_percent": round(gpu.memoryUsed/gpu.memoryTotal*100, 1)
        } for gpu in gpus]
    else:
        gpu_info = {"status": "No GPUs detected"}
except ImportError:
    gpu_info = {"status": "GPUtil not installed"}

# Combine all information
experiment_config = {
    "system_info": system_info,
    "key_packages": key_packages,
    "jupyter_info": jupyter_info,
    "hardware_info": hardware_info,
    "gpu_info": gpu_info
}

# Save to file for inclusion in research paper
config_file = 'experiment_configuration.json'
with open(config_file, 'w') as f:
    json.dump(experiment_config, f, indent=2)

# Display summary
print(f"Experiment configuration saved to {config_file}")
print("\nSummary:")
print(f"System: {system_info['platform']}")
print(f"Python version: {system_info['python_version'].split()[0]}")
print(f"Key packages: {', '.join([f'{k}={v}' for k,v in key_packages.items()][:3])}...")
if "memory_total_gb" in hardware_info:
    print(f"RAM: {hardware_info['memory_total_gb']} GB")
if isinstance(gpu_info, list) and len(gpu_info) > 0:
    print(f"GPU: {gpu_info[0].get('name', 'Unknown')}")


Experiment configuration saved to experiment_configuration.json

Summary:
System: Linux-5.14.0-284.86.1.el9_2.x86_64-x86_64-with-glibc2.35
Python version: 3.12.10
Key packages: numpy=1.26.4, pandas=2.2.3, matplotlib=3.10.3...
RAM: 377.07 GB


/state/partition1/job-60609498/ipykernel_2433177/1081351148.py:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [31]:
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
